In [ ]:
# example_solar_generation.py
## import needed packages
import pandas as pd
import pvlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from LESO import System
from LESO import PhotoVoltaic, PhotoVoltaicAdvanced, BifacialPhotoVoltaic
from LESO.feedinfunctions import PVlibweather
from LESO.feedinfunctions import bifacial

In [ ]:
## Set up the components and system
pv1 =           PhotoVoltaic('simplePV')
pv2 =           PhotoVoltaicAdvanced('advancedPV')
pv3 =           PhotoVoltaicAdvanced('trackingPV-NS', tracking=True)
pv4 =           PhotoVoltaicAdvanced('trackingPV2-EW', tracking=True, azimuth=-90)
pv5 =           BifacialPhotoVoltaic('bifacialPV')

modelname = 'Trial to compare'
system = System(52, 5, model_name = modelname)
system.add_components([pv1, pv2, pv3, pv4, pv5])
system.fetch_input_data()
system.calculate_time_series()

for pv in system.components:
    print(f'{pv.name}: {pv.state.power.sum()/pv.installed} kWh / ( year * kWp )')

# Note: bifacial not working ATM

In [ ]:
## plot yield over month
system.components[0].sum_by_month()

fig = plt.figure(figsize=(12,5))
i = 0.
r = np.arange(len(system.components[0].monthly_state.index))
barwidth = .2
colors = sns.color_palette('Set3', len(system.components))

for i, component in enumerate(system.components):
    component.sum_by_month()
    plt.bar(
        r, 
        list(component.monthly_state.values.flatten()), 
        label = component.name,
        width=barwidth,
        alpha = 1,
        color = colors[i]
    )
    plt.legend()
    r = [x + barwidth for x in r]

plt.xticks([x - 3*barwidth for x in r], list(system.components[0].monthly_state.index), rotation = -90)
plt.tick_params(left=False, labelleft=False) #remove ticks
plt.box(False) #remove box